In [14]:
import json
from neo4j import GraphDatabase
import numpy as np
import pandas as pd
from pprint import pprint
import time
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

## Establish connection to Neo4j

You could be running Neo4j in any variety of places.  This demo shows running it locally.  However, if you are hosting it somewhere else, replace the bolt address with yours.  Also update your password below.

In [3]:
URI = "bolt://localhost:7687"
USER = "neo4j"
PWD = "" # replace with your value
AUTH = (USER, PWD)

In [4]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
        

def insert_data(query, rows, batch_size=1000):

    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = neo_conn.query(query, 
                             parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result


neo_conn = Neo4jConnection(uri=URI, user=USER, pwd=PWD)

### Constraints

These uniqueness constraints are established because they speed up queries significantly.  You could also use a node key (Enterprise Edition only).

In [5]:
neo_conn.query('CREATE CONSTRAINT sg IF NOT EXISTS FOR (s:SGEntity) REQUIRE s.uid IS UNIQUE')
neo_conn.query('CREATE CONSTRAINT ppp IF NOT EXISTS FOR (p:PPPEntity) REQUIRE p.uid IS UNIQUE')
neo_conn.query('CREATE CONSTRAINT dol IF NOT EXISTS FOR (d:DOLEntity) REQUIRE d.uid IS UNIQUE')

[]

## Loading in the SafeGraph, PPP, and DOL data

I have put the data files in the same directory on my local machine as this notebook, but update your path to reflect your situation.

In this case, I am bringing in all of the data from the files EXCEPT in the case of the DOL Whisard data.  In that dataset, there are many columns detailing a multitude of types of violations.  I have chosen to only include the total violation count for brevity.

You will note that at the end I have created a column called "uid," which is the unique identifier for each node.  I have chosen to create this based on the data source (SAFEGRAPH, DOL_WHISARD, or PPP_LOANS) combined with the record ID.  (It should be noted that just the record ID is NOT unique, which is why I went this route.)  This arbitrary uid will speed up the MATCH queries in Cypher significantly.

In [8]:
sg_data = []

with open('./SafeGraph_LasVegas_Partial.json') as f:
    for line in f:
        sg_data.append(json.loads(line))

sg_df = pd.DataFrame(sg_data)

dol_data = []

with open('./Dept_Labor_Whisard_LasVegas.json') as f:
    for line in f:
        dol_data.append(json.loads(line))

dol_df = pd.DataFrame(dol_data)
dol_df = dol_df.iloc[:, :13]  # Each individual infraction count is shown beyond column 13

ppp_data = []

with open('./PPP_Loans_Over_150k_LasVegas.json') as f:
    for line in f:
        ppp_data.append(json.loads(line))

ppp_df = pd.DataFrame(ppp_data)

dol_df.replace('null', np.nan, inplace=True)

sg_df.fillna('Not Provided', inplace=True)
ppp_df.fillna('Not Provided', inplace=True)
dol_df.fillna('Not Provided', inplace=True)

sg_df["name"] = sg_df["LOCATION_NAME_ORG"]
ppp_df["name"] = ppp_df["BUSINESS_NAME_ORG"]
dol_df["name"] = dol_df["BUSINESS_NAME_ORG"]

ppp_df["address"] = ppp_df["BUSINESS_ADDR_LINE1"].astype(str) + " " + ppp_df["BUSINESS_ADDR_CITY"].astype(str) + " " + ppp_df["BUSINESS_ADDR_STATE"].astype(str) + " " + ppp_df["BUSINESS_ADDR_POSTAL_CODE"].astype(str)

dol_df["address"] = dol_df["BUSINESS_ADDR_LINE1"].astype(str) + " " + dol_df["BUSINESS_ADDR_CITY"].astype(str) + " " + dol_df["BUSINESS_ADDR_STATE"].astype(str) + " " + dol_df["BUSINESS_ADDR_POSTAL_CODE"].astype(str)

sg_df["uid"] = sg_df["DATA_SOURCE"] + '.' + sg_df["RECORD_ID"].astype(str)
ppp_df["uid"] = ppp_df["DATA_SOURCE"] + '.' + ppp_df["RECORD_ID"].astype(str)
dol_df["uid"] = dol_df["DATA_SOURCE"].str.upper() + '.' + dol_df["RECORD_ID"].astype(str)

sg_df.shape, dol_df.shape, ppp_df.shape

((79946, 27), (1554, 16), (3488, 22))

In [9]:
sg_df.head()

,DATA_SOURCE,RECORD_ID,RECORD_TYPE,PLACEKEY,REL_ANCHOR_DOMAIN,REL_ANCHOR_KEY,LOCATION_NAME_ORG,BRANDS,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,BUSINESS_GEO_LATITUDE,BUSINESS_GEO_LONGITUDE,CATEGORY_TAGS,CLOSED_ON,TRACKING_CLOSED_SINCE,PHONE_NUMBER,BUSINESS_ADDR_COUNTRY,BUSINESS_ADDR_FULL,MAILING_VERIFIED_STATUS,REL_POINTER_DOMAIN,REL_POINTER_KEY,REL_POINTER_ROLE,OPENED_ON,IS_INTERSECTION,name,uid
0,SAFEGRAPH,225-222@5yv-j92-tn5,ORGANIZATION,225-222@5yv-j92-tn5,PLACEKEY,225-222@5yv-j92-tn5,Cantwell Michelle L Atty,[],Legal Services,Offices of Lawyers,541110,36.145647,-115.186399,[],2024-02-01,2019-07-01,+17023627800,US,3320 W Sahara Ave Las Vegas NV 89102-3223,VERIFIED_PREMISE,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,Cantwell Michelle L Atty,SAFEGRAPH.225-222@5yv-j92-tn5
1,SAFEGRAPH,226-222@5yv-hmm-whq,ORGANIZATION,226-222@5yv-hmm-whq,PLACEKEY,226-222@5yv-hmm-whq,Pieology Pizzeria,"[ { ""safegraph_brand_id"": ""SG_BRAND_f372f9969f...",Restaurants and Other Eating Places,Limited-Service Restaurants,722513,36.144906,-115.332644,"[ ""Counter Service"", ""Dinner"", ""Fast Food"", ""L...",Not Provided,2019-07-01,+17023314454,US,10965 Lavender Hill Dr Ste 130 Las Vegas NV 89...,VERIFIED_DELIVERY_POINT,PLACEKEY,zzw-223@5yv-hkm-rc5,PARENT,Not Provided,Not Provided,Pieology Pizzeria,SAFEGRAPH.226-222@5yv-hmm-whq
2,SAFEGRAPH,22s-222@5yv-jbz-jgk,ORGANIZATION,22s-222@5yv-jbz-jgk,PLACEKEY,22s-222@5yv-jbz-jgk,Ellen Peneyra,[],Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,621399,36.10289,-115.121807,[],Not Provided,2019-07-01,+17027397716,US,2275 Renaissance Dr Ste D Las Vegas NV 89119-6797,VERIFIED_DELIVERY_POINT,PLACEKEY,22g-222@5yv-jbz-h89,PARENT,Not Provided,Not Provided,Ellen Peneyra,SAFEGRAPH.22s-222@5yv-jbz-jgk
3,SAFEGRAPH,28m-222@5yv-hmp-v4v,ORGANIZATION,28m-222@5yv-hmp-v4v,PLACEKEY,28m-222@5yv-hmp-v4v,Stanford Jackson,[],Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,621399,36.143386,-115.272694,[],Not Provided,2019-07-01,+17025623569,US,8321 W Sahara Ave Apt 2007 Las Vegas NV 89117-...,VERIFIED_DELIVERY_POINT,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,Stanford Jackson,SAFEGRAPH.28m-222@5yv-hmp-v4v
4,SAFEGRAPH,228-232@5yv-hts-bp9,ORGANIZATION,228-232@5yv-hts-bp9,PLACEKEY,228-232@5yv-hts-bp9,MS. Jazlyn James,[],Offices of Physicians,"Offices of Physicians, Mental Health Specialists",621112,36.261833,-115.255012,[],Not Provided,2019-07-01,+17029008666,US,5550 Painted Mirage Rd Ste 320 Las Vegas NV 89...,VERIFIED_DELIVERY_POINT,PLACEKEY,223-23b@5yv-hts-bp9,PARENT,Not Provided,Not Provided,MS. Jazlyn James,SAFEGRAPH.228-232@5yv-hts-bp9


In [10]:
dol_df.head()

,RECORD_TYPE,DATA_SOURCE,RECORD_ID,case_id,BUSINESS_NAME_ORG,LEGAL_NAME_ORG,BUSINESS_ADDR_LINE1,BUSINESS_ADDR_CITY,BUSINESS_ADDR_STATE,BUSINESS_ADDR_POSTAL_CODE,naic_cd,naics_code_description,case_violtn_cnt,name,address,uid
0,ORGANIZATION,DoL_WHISARD,53,1658108,Fabulous Freddy's (Trailwood),Fabulous LLC,9611 Trail Wood Drive,Las Vegas,NV,89134,811192,Car Washes,0,Fabulous Freddy's (Trailwood),9611 Trail Wood Drive Las Vegas NV 89134,DOL_WHISARD.53
1,ORGANIZATION,DoL_WHISARD,165,1419689,Boulder Station Hotel & Casino,"Boulder Station, Inc.",4111 Boulder Hwy,Las Vegas,NV,89121,721120,Casino Hotels,1,Boulder Station Hotel & Casino,4111 Boulder Hwy Las Vegas NV 89121,DOL_WHISARD.165
2,ORGANIZATION,DoL_WHISARD,178,1424856,MSI Landscaping,"MIST Systems International, Inc.","4820 Quality Court, #B",Las Vegas,NV,89103,561730,Landscaping Services,14,MSI Landscaping,"4820 Quality Court, #B Las Vegas NV 89103",DOL_WHISARD.178
3,ORGANIZATION,DoL_WHISARD,196,1668849,Pastime Pools,Pastime Pools,731 Memory Ln,Las Vegas,NV,89110,238990,All Other Specialty Trade Contractors,0,Pastime Pools,731 Memory Ln Las Vegas NV 89110,DOL_WHISARD.196
4,ORGANIZATION,DoL_WHISARD,491,1602023,Steven Michaels,Steven Michaels,4108 Autum St,Las Vegas,NV,89120,8141,Private Households,2,Steven Michaels,4108 Autum St Las Vegas NV 89120,DOL_WHISARD.491


In [11]:
ppp_df.head()

,RECORD_TYPE,DATA_SOURCE,RECORD_ID,Loan_Range,BUSINESS_NAME_ORG,BUSINESS_ADDR_LINE1,BUSINESS_ADDR_CITY,BUSINESS_ADDR_STATE,BUSINESS_ADDR_POSTAL_CODE,NAICS_Code,Business_Type,OwnedByRaceEthnicity,OwnedBy,OwnedByVeteran,NonProfit,JobsReported,DateApproved,Lender,CD,name,address,uid
0,ORGANIZATION,PPP_LOANS,7017,c $1-2 million,"INFINITY HOSPICE CARE OF LAS VEGAS, LLC",5110 N 40TH ST STE 107,PHOENIX,AZ,85018,623110,Limited Liability Company(LLC),Unanswered,Male Owned,Unanswered,,137,05/01/2020,"JPMorgan Chase Bank, National Association",AZ-09,"INFINITY HOSPICE CARE OF LAS VEGAS, LLC",5110 N 40TH ST STE 107 PHOENIX AZ 85018,PPP_LOANS.7017
1,ORGANIZATION,PPP_LOANS,7018,"d $350,000-1 million",CLUB TATTOO LAS VEGAS LLC,1839 s. almaschool rd. ste 230,MESA,AZ,85210,812199,Limited Liability Company(LLC),Unanswered,Male Owned,Non-Veteran,,15,05/01/2020,"JPMorgan Chase Bank, National Association",AZ-09,CLUB TATTOO LAS VEGAS LLC,1839 s. almaschool rd. ste 230 MESA AZ 85210,PPP_LOANS.7018
2,ORGANIZATION,PPP_LOANS,7021,"d $350,000-1 million","LAS VEGAS LABOR, LLC","10265 W Camelback Rd, Ste 104",PHOENIX,AZ,85037,111421,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,,0,04/07/2020,"UMB Bank, National Association",AZ-03,"LAS VEGAS LABOR, LLC","10265 W Camelback Rd, Ste 104 PHOENIX AZ 85037",PPP_LOANS.7021
3,ORGANIZATION,PPP_LOANS,7022,"e $150,000-350,000","FIRST CUP PARTNERS LAS VEGAS II, LLC","106 S Kyrene Rd, Ste 2",CHANDLER,AZ,85226,722511,Limited Liability Company(LLC),Unanswered,Male Owned,Non-Veteran,,105,04/16/2020,"JPMorgan Chase Bank, National Association",AZ-09,"FIRST CUP PARTNERS LAS VEGAS II, LLC","106 S Kyrene Rd, Ste 2 CHANDLER AZ 85226",PPP_LOANS.7022
4,ORGANIZATION,PPP_LOANS,7023,"e $150,000-350,000","FIRST CUP PARTNERS LAS VEGAS, LLC",2121 W Chandler Blvd Ste 106,Chandler,AZ,85224,,Limited Liability Company(LLC),Unanswered,Male Owned,Non-Veteran,,124,05/01/2020,"JPMorgan Chase Bank, National Association",AZ-09,"FIRST CUP PARTNERS LAS VEGAS, LLC",2121 W Chandler Blvd Ste 106 Chandler AZ 85224,PPP_LOANS.7023


## Functions for loading Pandas dataframes into Neo4j

Paco Nathan's article shows how to do this using the Neo4j Graph Data Science (GDS) python library.  I decided to show it this way so that there are multiple different approaches documented on the internet.  (This is also similar to my approach on previous blog posts, so just trying to be consistent!)

In [12]:
def add_sg(rows, batch_size=10000):

    query = """UNWIND $rows AS row
               MERGE (:SGEntity {uid: row.uid,
                               record_id: row.RECORD_ID,
                               data_source: row.DATA_SOURCE,
                               record_type: row.RECORD_TYPE,
                               brands: row.BRANDS,
                               top_category: row.TOP_CATEGORY,
                               sub_category: row.SUB_CATEGORY,
                               naics_code: row.NAICS_CODE,
                               category_tags: row.CATEGORY_TAGS,
                               address: row.BUSINESS_ADDR_FULL,
                               mailing_verified_status: row.MAILING_VERIFIED_STATUS,
                               name: row.name})
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_ppp(rows, batch_size=10000):

    query = """UNWIND $rows AS row
               MERGE (:PPPEntity {uid: row.uid,
                               record_id: row.RECORD_ID,
                               record_type: row.RECORD_TYPE,
                               data_source: row.DATA_SOURCE,
                               loan_range: row.Loan_Range,
                               address: row.address,
                               name: row.name,
                               naics_code: row.NAICS_Code,
                               business_type: row.Business_Type,
                               jobs_reported: row.JobsReported,
                               date_approved: row.DateApproved,
                               lender: row.Lender})
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_dol(rows, batch_size=10000):

    query = """UNWIND $rows AS row
               MERGE (:DOLEntity {uid: row.uid,
                               record_id: row.RECORD_ID,
                               record_type: row.RECORD_TYPE,
                               data_source: row.DATA_SOURCE,
                               case_id: row.case_id,
                               name: row.name,
                               legal_name_org: row.LEGAL_NAME_ORG,
                               address: row.address,
                               naics_code: row.naic_cd,
                               naics_code_description: row.naics_code_description,
                               case_violtn_cnt: row.case_violtn_cnt})
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)

In [15]:
add_sg(sg_df)

{'total': 10000, 'batches': 1, 'time': 1.1862220764160156}
{'total': 20000, 'batches': 2, 'time': 2.439103364944458}
{'total': 30000, 'batches': 3, 'time': 3.6555025577545166}
{'total': 40000, 'batches': 4, 'time': 4.83707594871521}
{'total': 50000, 'batches': 5, 'time': 6.155801773071289}
{'total': 60000, 'batches': 6, 'time': 7.4046595096588135}
{'total': 70000, 'batches': 7, 'time': 8.666188955307007}
{'total': 79946, 'batches': 8, 'time': 9.929372787475586}


{'total': 79946, 'batches': 8, 'time': 9.929372787475586}

In [16]:
add_dol(dol_df)

{'total': 1554, 'batches': 1, 'time': 0.12855982780456543}


{'total': 1554, 'batches': 1, 'time': 0.12855982780456543}

In [17]:
add_ppp(ppp_df)

{'total': 3488, 'batches': 1, 'time': 0.3720555305480957}


{'total': 3488, 'batches': 1, 'time': 0.3720555305480957}